## Glassnode Endpoint Scrape and API Call
Glassnode contains many useful metrics for bitcoin and ethereum.  They have an API to access these endpoints, but only one request can be made at a time and only so many requests can be made a minute.  Additionaly, a link must be copied and pasted from the API documentation to the code to call the API for each metric.  With nearly 300 metrics, this is a cumbersome process.  

The goal of this task is to scrape all of the endpoints from the glassnode API documentation and identify wether they are for BTC or ETH.  Then, call the API for each of the coin's endpoints at a rate that will not terminate the connection.  

The output of this function is a dataset with daily coin metrics going back to 2010.  This dataset can be used for machine learning algorithms to learn and predict future BTC prices.

In [4]:
import numpy as np
import datetime as dt
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import re

pd.set_option('display.max_colwidth', -1)

<ipython-input-4-60bc3006f94c>:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [5]:
#define function to convert date to unix
def date_to_unix(
        year=2010,
        month=1,
        day=15,
):
    time = int((dt.datetime(year, month, day, 0, 0, 0).timestamp()))
    return time

In [7]:
#Endpoint Scraping

#Create list of urls to scrape for endpoints
urls = {}
coin = {}
pages = {'addresses' : 'https://docs.glassnode.com/api/addresses',
         'blockchain' : 'https://docs.glassnode.com/api/blockchain',
         'derivatives' : 'https://docs.glassnode.com/api/derivatives', 
         'distribution' : 'https://docs.glassnode.com/api/distribution',
         'entities' : 'https://docs.glassnode.com/api/entities',
         'eth2' : 'https://docs.glassnode.com/api/eth2',
         'fees' : 'https://docs.glassnode.com/api/fees',
         'indicators' : 'https://docs.glassnode.com/api/indicators',
         'institutions' : 'https://docs.glassnode.com/api/institutions',
         'market' : 'https://docs.glassnode.com/api/market',
         'mining' : 'https://docs.glassnode.com/api/mining',
         'protocols' : 'https://docs.glassnode.com/api/protocols',
         'defi' : 'https://docs.glassnode.com/api/defi',
         'supply' : 'https://docs.glassnode.com/api/supply',
         'transactions' : 'https://docs.glassnode.com/api/transactions'
         }
#access each url and scrape for endpoints 
for page in pages.keys():
    response = requests.get(pages[page])
    content = response.content
    parser = BeautifulSoup(content, 'html.parser')
    text = parser.text
    #extract endpoint
    string = r"(/v1/metrics/[a-z]+/\S+[a-z\d])[A-Z]"
    string2 = r"a=([A-Z]{3})&m=[a-z]+"        
    urls[page] = re.findall(string, text)
    coin[page] = re.findall(string2, text)

#place endpoints in dictionary with appropriate variable name
endpoints = {}
for page in pages.keys():
    for index, url in enumerate(urls[page]):
        var = re.findall(r"/v1/metrics/[a-z]+/(.+)", url)[0]
        endpoints[var] = [url, coin[page][index]]
urls = [endpoints[x][0] for x in endpoints.keys()]
coins = [endpoints[x][1] for x in endpoints.keys()]
var = endpoints.keys()
endpoints = pd.DataFrame({'feature' : var, 'coin' : coins, 'urls' : urls})
#make correction for 'adjusted90' urls (remove 90 from url)
endpoints['urls'] = endpoints['urls'].str.replace('adjusted90', 'adjusted')


In [10]:
endpoints[endpoints['feature'].str.contains('price_usd_ohlc')]

,feature,coin,urls
199,price_usd_ohlc,BTC,/v1/metrics/market/price_usd_ohlc


In [1]:
import requests as req
from datetime import date
today = date.today()

#Define function to access API and request data
def get_metrics(coin, endpoints = endpoints, start = [2010, 9, 1], until = [today.year, today.month, today.day]):
        #filter to coins
        endpoints = endpoints[endpoints['coin'] == coin]
        #Initiate API parameters and dictionary for each coins response
        coins_data = {}
        
        #call api
        api_key='1t0KVebVMNMdGk149JtLxFIfBFk',  # Insert your own API key here, this one is only illustrative.
        asset=coin,
        status=False,
        headers=False,
        resolution='24h',
        wait=10
        
        # convert dates to unix
        s = date_to_unix(year=start[0], month=start[1], day=start[2])
        u = date_to_unix(year=until[0], month=until[1], day=until[2])
        
        #loop through metrics and request data.  Store in dictionary
        for index, row in endpoints.iterrows():

            feature = row[0]
            url = row[2]
            response = req.get(
                f'https://api.glassnode.com{url}',
                {
                    'api_key': api_key,
                    'a': asset,
                    's': s,
                    'u': u,
                    'i': resolution
                },
                timeout=wait)
            print(index, feature)
            try:
                response.json()[0]['o'] 
                print('nested')
                data = response.json()
                result = pd.json_normalize(data)
            except KeyError:
                print('not nested')
                result= pd.DataFrame.from_dict(response.json())
            #Intitiate dataframe with time column
            if index == 0:
                coin_response = pd.to_datetime(result['t'], unit='s')
            #rename columns
            col_dict = {x:'{}_{}'.format(feature, x) for x in result.columns}
            result = result.rename(columns = col_dict)   
            #print status and pause
            if status:
                print(response)
            time.sleep(2)
            #concat data            
            coin_response = pd.concat([coin_response, result.iloc[:,1:]], axis=1)
        return coin_response

NameError: name 'endpoints' is not defined

In [ ]:
#define coin
coin = 'BTC'
data = get_metrics(coin)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4018 entries, 0 to OKEx
Columns: 449 entries, t to transfers_volume_from_exchanges_sum_v
dtypes: datetime64[ns](1), float64(428), object(20)
memory usage: 13.8+ MB


In [24]:
data.select_dtypes(include = [object]).shape

(4018, 20)

In [25]:
data.select_dtypes(include = [object])

,options_open_interest_distribution_data,options_open_interest_distribution_exchange,balance_miners_all_balances,hash_ribbon_o.ma30,hash_ribbon_o.ma60,difficulty_ribbon_o.ma128,difficulty_ribbon_o.ma14,difficulty_ribbon_o.ma200,difficulty_ribbon_o.ma25,difficulty_ribbon_o.ma40,difficulty_ribbon_o.ma60,difficulty_ribbon_o.ma9,difficulty_ribbon_o.ma90,spent_output_price_distribution_ath_partitions,spent_output_price_distribution_percent_partitions,utxo_realized_price_distribution_ath_partitions,utxo_realized_price_distribution_percent_partitions,difficulty_latest_v,hash_rate_mean_v,transfers_volume_miners_to_exchanges_all_volumes
0,"{'contract_expiration': 1613433600, 'calls': [{'strike': 42000, 'value': 0}, {'strike': 43000, 'value': 0}, {'strike': 44000, 'value': 0}, {'strike': 45000, 'value': 0.1}, {'strike': 46000, 'value': 2.8}, {'strike': 47000, 'value': 31.3}, {'strike': 48000, 'value': 72.1}, {'strike': 49000, 'value': 69}, {'strike': 50000, 'value': 89.5}, {'strike': 51000, 'value': 64.6}, {'strike': 52000, 'value': 56.2}, {'strike': 53000, 'value': 95}, {'strike': 54000, 'value': 39.3}, {'strike': 55000, 'value': 12.1}, {'strike': 56000, 'value': 14.3}], 'puts': [{'strike': 42000, 'value': 51.9}, {'strike': 43000, 'value': 30.4}, {'strike': 44000, 'value': 66.5}, {'strike': 45000, 'value': 64.9}, {'strike': 46000, 'value': 71.3}, {'strike': 47000, 'value': 60.7}, {'strike': 48000, 'value': 53}, {'strike': 49000, 'value': 2.9}, {'strike': 50000, 'value': 2.4}, {'strike': 51000, 'value': 0}, {'strike': 52000, 'value': 0}, {'strike': 53000, 'value': 0}, {'strike': 54000, 'value': 0}, {'strike': 55000, 'value': 0}, {'strike': 56000, 'value': 0}]}",Deribit,"{'Genesis': 50, 'Patoshi': 1096245.49, 'other': 1387903.0300007}",4183103398.500436,2767833571.969928,640108044667.162598,2437738598990.179688,417955787656.983582,2140327555491.711182,1745622731116.665527,1292012114786.571289,2570898602545.585449,886936592699.89978,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 711.789999999999, 0, 0.5, 0, 0, 175, 0, 100.01, 0, 510, 0.05, 3522.61, 2761.1899985, 1273.65, 3166.44, 5070.6899988, 862.0400001, 9788, 100, 0.03, 73.05, 1.98, 1, 0, 39.5, 500.03, 0, 0, 0, 0, 0, 0, 0, 50.03, 50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 711.789999999999, 0.5, 0, 0, 175, 100.01, 0, 510, 3522.64, 2761.2099985, 1283.65, 3206.45, 5882.7199989, 9788, 100.02, 73.06, 1.98, 1, 39.5, 500.03, 0, 0, 0, 0, 0, 50.03, 50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 150, 0, 370.17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3179460.08, 9336.15, 20353.94, 80, 0, 20733.17, 3536.42, 4633.13, 5739.55, 8752.16, 29781.76, 41799.9699987, 104257.2000001, 21502.8400001, 14058.1000001, 88411.1400006, 99884.1000004, 43447.68, 43304.02, 28935.82, 14200.08, 10308.98, 32696.79, 14062.62, 4423.12, 6338.83, 100.3, 3946.96, 1452.25, 2095.8, 150.15, 885.68, 1805.12, 4483.86, 2756.77, 0, 0, 200, 0, 3132.91, 0, 100.2, 0, 0, 8632.07, 0, 200.11]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 150, 0, 370.17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3171706.5, 17089.73, 20353.94, 80, 20733.17, 8169.55, 5589.45, 8652.03, 33436.7999985, 83255.3200003, 80559.3500001, 22589.6600001, 102920.8800007, 120631.0100003, 43724.62, 42465.21, 8773.67, 40391.71, 12576.22, 6338.83, 1992.26, 2155.05, 3448, 150.15, 2690.8, 4483.86, 2756.77, 0, 200, 3132.91, 0, 100.2, 0, 7269.69, 1362.38, 200.11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",2677426605389.910156,4934045155.77458,{'other': 7.08243195898143}
1,"{'contract_expiration': 1613520000, 'calls':